In [1]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
import csv

csv_files = ['./ADNI1_Annual_2_Yr_3T_12_05_2022.csv', 'ADNI1_Complete_1Yr_3T_12_12_2022.csv']

# Ex: {'037_S_0303': 'CN'}
patient_groups = {}
for file_path in csv_files:
    with open(file_path, 'r') as file:
        csvreader = csv.reader(file)
        next(csvreader)
        for row in csvreader:
            if(row[2] == 'MCI'): continue
            patient_groups[row[1]] = row[2]

In [3]:
IMG_SHAPE = (100, 128, 128)

def resample_img(itk_image, out_spacing=[2.0, 2.0, 2.0]):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] *
            (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] *
            (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize((IMG_SHAPE[2], IMG_SHAPE[1], IMG_SHAPE[0]))
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)


In [4]:
def slices_matrix_2D(img):
    images = []

    TOP = math.floor(IMG_SHAPE[1] * 0.7)
    N_CUTS = 24
    STEP = math.floor((TOP - IMG_SHAPE[1] * 0.4) / N_CUTS)

    cut_it = TOP

    for cutting_time in range(N_CUTS):
        cut = img[:IMG_SHAPE[1], cut_it, :IMG_SHAPE[2]]
        images.append(cut)

        cut_it -= STEP

    return np.array(images)

In [5]:
import os

SKULL_STRIPPED_DB = '/home/marlon/escola/faculdade/6/padroes/ADNI_SKULL_STRIPPED/'

def skull_strip_nii(original_img, destination_img, frac=0.4):
    cmd_robustfov = f'robustfov -i {original_img} -r {destination_img}.cropped'
    cmd_bet = f'bet {destination_img}.cropped {destination_img} -f {frac} -A -B -R'

    os.system(cmd_robustfov)
    os.system(cmd_bet)


In [6]:
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

rootdir = '/home/marlon/escola/faculdade/6/padroes/ADNI'

data = []
labels = []

count = 0
for subdir, dirs, files in os.walk(rootdir):
    relative_path = subdir[len(rootdir):]
    if (len(relative_path) == 0):
        continue
    patient = relative_path.rsplit('/')[1]

    if patient in patient_groups:
        group = patient_groups[patient]
        # print(group)
        for file in files:
            try:
                file_path = os.path.join(subdir, file)
                skull_stripped_file = os.path.join(SKULL_STRIPPED_DB, file)
                skull_strip_nii(file_path, skull_stripped_file)

                sitk_image = sitk.ReadImage(skull_stripped_file)
                sitk_image = resample_img(sitk_image)
                img = sitk.GetArrayFromImage(sitk_image)
                img = (img - np.min(img)) / (np.max(img) - np.min(img)) # Normalizar entre 0 e 1
                images = slices_matrix_2D(img)

                # print(os.path.join(subdir, file))
                # print(images.shape)
                # for image in images:
                #     plt.imshow(image, cmap='gray')
                #     plt.show()

                data.extend(images)
                labels.extend([group] * len(images))

                # print(np.array(data).shape)
                # print(np.array(labels).shape)
                # print(np.array(data))
            except Exception as e:
                print(f'Error on {file}: {e}')

            count+=1
            print(count)

with open('data.npy', 'wb') as f:
    np.save(f, np.array(data))

with open('labels.npy', 'wb') as f:
    np.save(f, np.array(labels))


Final FOV is: 
34.000000 165.000000 0.000000 256.000000 0.000000 166.000000 



/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/bet: line 145: /bin/fslstats: No such file or directory


1
Final FOV is: 
30.000000 165.000000 0.000000 256.000000 0.000000 166.000000 



/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/bet: line 145: /bin/fslstats: No such file or directory


2
Final FOV is: 
1.000000 165.000000 0.000000 256.000000 0.000000 166.000000 



...

268
Final FOV is: 
70.000000 169.000000 0.000000 256.000000 0.000000 160.000000 



/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/bet: line 145: /bin/fslstats: No such file or directory


269
Final FOV is: 
51.000000 169.000000 0.000000 256.000000 0.000000 160.000000 



/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/fslval: line 91: /fsl/bin/fslhd: No such file or directory
/bin/bet: line 145: /bin/fslstats: No such file or directory


270


In [7]:
# IMAGE = '/home/marlon/escola/faculdade/6/padroes/ADNI/002_S_0413/MPR____N3__Scaled/2006-05-19_16_17_47.0/I40657/ADNI_002_S_0413_MR_MPR____N3__Scaled_Br_20070216232854688_S14782_I40657.nii'
# IMAGE2 = '/home/marlon/escola/faculdade/6/padroes/ADNI/002_S_0413/MPR____N3__Scaled/2006-05-19_16_17_47.0/I40657/teste.nii.gz'

# # load in sitk format (SimpleITK)
# sitk_image = sitk.ReadImage(IMAGE)
# sitk_image2 = sitk.ReadImage(IMAGE2)
# # transform into a numpy array
# img = sitk.GetArrayFromImage(sitk_image)
# img2 = sitk.GetArrayFromImage(sitk_image2)

# plt.imshow(img[:,:,70], cmap='gray')
# plt.show()

# plt.imshow(img2[:,:,70], cmap='gray')
# plt.show()

In [8]:
data = np.array(data)
labels = np.array(labels)

# for img_ex in data[labels == 'AD']:
#     plt.imshow(img_ex, cmap='gray')
#     plt.show()